In [185]:
import requests
import lxml.html as lh
import pandas as pd


url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)



In [186]:
tr_elements =doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [187]:
for j in range(1, len(tr_elements)):
    T = tr_elements[j]
    if len(T)!=3:
        break
       
    i=0
    for t in T.iterchildren():
        data=t.text_content()
        if i>0:
              try:
                data =int(data)
              except:
                 pass
        col[i][1].append(data)
        i+=1
#[len(C) for (title, C) in col

In [188]:
Dict={title:column for (title,column) in col}


df=pd.DataFrame(Dict)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [189]:
cols=df.columns.tolist()
cols


['Borough', 'Neighbourhood\n', 'Postcode']

In [190]:
cols = cols[-1:] + cols[:-1]
df=df[cols]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [191]:
import numpy as np

df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned\n
1,M2A,NaN,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [192]:
missing_data =df.isnull()
missing_data.head()

,Postcode,Borough,Neighbourhood
0,False,True,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


In [193]:
df.dropna(subset=["Borough"],axis=0, inplace=True)

In [194]:
df.reset_index(drop=True, inplace=True)

In [195]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [196]:
df_postcode=df.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(','.join).reset_index()
df_postcode.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


In [198]:
df_postcode.rename(columns={'Neighbourhood\n': 'Neighbourhood'})
df=df_postcode.replace('\n','', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
